In [3]:
from altProts_in_communities.utils import *
from altProts_in_communities.parse_psms import *

In [4]:
if os.path.exists('OP16_bioplex_all_psms.pkl'):
    bioplex_psms = pickle.load(open('OP16_bioplex_all_psms.pkl', 'rb'))
else:
    with Pool(10) as p:
        all_psms = p.map(get_all_psms, all_baits)

    bioplex_psms = {}
    for bait_psms in all_psms:
        bioplex_psms.update(bait_psms)

    pickle.dump(bioplex_psms, open('OP16_bioplex_all_psms.pkl', 'wb'))

In [5]:
all_alt_psms = []
alt_peptides = []
for bait, bait_exps in bioplex_psms.items():
    for exp_id, exp_dict in bait_exps['exps'].items():
        for prey, prey_dict in exp_dict['psms'].items():
            if 'IP_' in prey:
                all_alt_psms.extend(prey_dict['psms'])
                for pep, spec in prey_dict['psms']:
                    alt_peptides.append((prey, bait, pep))

unique_altpeps = list(set([x[-1] for x in alt_peptides]))
print('count unique peptides:', len(unique_altpeps))
print('count altProts:', len(set(x[0] for x in alt_peptides)))
print('count altProt genes:', len(set(x[0].split('|')[0] for x in alt_peptides)))
with open('peptides_for_nextprot.txt', 'w') as f:
    f.write(', '.join(unique_altpeps))

count unique peptides: 910
count altProts: 426
count altProt genes: 414


In [6]:
# nextprot rejects
nexprot_rejects = "KALSKQEMVSASSSQRGR,QHHCPNTPIILVGTK,LLQDFINGK,MHAQDFTVSAMHGDMDQK,SYSPYDMLESIKK,NTLTGKTITLEVEPSDTIENVK,PIEEIIVQDGK,AVSLSLPR,ALTFWNEEIVPQIK,ILLLAELR,\
IACTGEKLDEREAGITEK,MLNKPIEEIIVQDGK,IHTGEKPNICEECGK,IINKPTAAAIAYGLDK,SGGTLVLVGLGSEMTTVPLLHAA,ESTLHLVLRR,DGNASGTMLLEALDCILPPTRPTDKPLR,LTHSLGWGTGSGMGTLLISK,\
MKIFVKTLTGKTITLEVEPSDTIENVK,KPMQFLGDEEMVRKAMEAVAAQGK,LLELVGQSLLR,VFIEDVIR,DCRATWVVLGHSER,LGLDVEIATYRK,TTGIVMDSGDGFTHTVPIYEGNALPH,PDNVTFEEGALIDPLSVGIHACRR,IINDTENLVR,\
NQLTSNPKNTVFDAK,DVDEVSSLLR,LQQESTFLFHGSKTEDTPDKK,DLQNVNITLRIIFQPVASQLPR,SFNCMPIEGMLSHQLK,DVVLHQTSGHIHNK,ETPVTHLYFGR,IQDEEGIPPDQQR,QQGKVEIIANDQGNR,LGLDVEIATYR,NKPIEEIIVQDGK,\
KTDPSSLGATSASFNFGEK,QYQEVMNSK,SGGTLVLVGLGSEMTTVPLLH,QLEDGRSLSDYNIQKESTLHLVLR,QSKPVTTPEEIAQVAMISANGDK,LEAIREDFLHER,SVMEEVLIPDEDYSLDEIHTEVSK,LAELEDALQK,VRLELTELEAIREDFLHER,\
YPIEHSIITNWDDMEK,TLVLVGLGSEMTTVPLLHAAIR,TLSNYNIQKESTLHLVLR,SGGTLVLVGLGSEMTTVPLLHAAIR,LTADLSAETLQAR,RLTQLSVMEEVLIPDEDYSLDEIHTEVSK,DGNASGTMLLEALDCILPPTRPTDKPLR,EETPVTHLYF,\
LTHSLGWGTGSGMGTLLISK,MRKAMEAVAAQGK,HISEQFTAMFR,STLHLVLRR,DGNASGTMLLEALDCILPPTRPTDK,LRIIFQPVASQLPR,VWFPDVMPL,EDYSLDEIHTEVSK,TFWNEEIVPQIK,ISISFGDLNPFPVR,ESTLHLVLRR,\
GVGISVLEMSHRPSDFAKI,KSGDAAIADMVPGKPMCVESFSDYPPLGR,RALTFWNEEIVPQIK,SGGTLVLVGLGSEMTTVPL,LELTELEAIREDFLHER,EETPVTHLYFGR,MISANGDKEIGNIISDAMK,LYEEEFETFFK,\
GTLVLVGLGSEMTTVPLLHAAIR,EDQTEYLEERK,KPTAAAIAYGLDK,VSEIHMTGPMCLIENTNGR,FQPVASQLPR,LGLDVEIATYRK,QQLDGCVTKCVDDHMHLIPTMTK,SGDAAIADMVPGKPMCVESFSDYPPLGR,IINDTENLVR,IINNAENLVR,\
TPVTHLYFGR,ADDAGAVGGPGGPGGPGMGNR,RHLPVFTYTDER,FGDLNPFPVR,HLPVFTYTDER,LPVFTYTDER,PRPLLLGR,MLNKPIEEIIVQDGK,GILGYTEHQVVSSDFNSNTHS,ADDAGAVGGPGGPGGPGMGNRGGFR,\
AKWYPEVQHHCPNTPIILVGTK,GCVTKCVDDHMHLIPTMTK,LLLVSFDSFR,SSNELVDDLFK,QTLAVGVIK,LPDNVTFEEGALIDPLSVGIHACRR,SGGTLVLVGLGSEMTTVPLLHAAIREVDIK,PIEEIIVQDGK,ALSKQEMVSASSSQRGR,\
LTQLSVMEEVLIPDEDYSLDEIHTEVSK,LISISFGDLNPFPVR,RSGGTLVLVGLGSEMTTVPLLHAAIR,VAHLIEWKGWSAQR,NTLTGKTITLEVEPSDTIENVK,ILLLAELR,HQGVMVGIGQK,ALTFWNEEIVPQIK,FGTINIIHPK,\
NQLTSNPKNTVFDAK,PVTHLYFGR,AVSLSLPR,IIFQPVASQLPR,SGPFGQIYRPDNFVFGQSGAGNNWAK,THINIVVIGHVNSGK,GVVVGDGAVGK,KDGNASGTMLLEALDCILPPTRPTDKPLR,NKPIEEIIVQDGK,LAELEDALQK,\
SFIIKLQR,VEKSYELPDGQVITIGNER,RIIFQPVASQLPR,LGLDVEIATYR,IVSNTSCTTNCLAPLAK,FWEVISDK,VFIEDVIR,EEMVRKAMEAVAAQGK,EWGPIFNILK"

# isobaric
exclude_peptides = {'NKPIEEIIVQDGK' ,'PIEEIIVQDGK', 'MLNKPIEEIIVQDGK', 'IINDTENLVR'}

# saav peptides not relevant in HEK293T
rescued_saav_peptides = pickle.load(open('rescued_saav_peptides.pkl', 'rb'))

nexprot_rejects = set(nexprot_rejects.split(',')).union(exclude_peptides).difference(rescued_saav_peptides)
print('count peptides rejected:', len(nexprot_rejects))

count peptides rejected: 18


In [7]:
all_pairs, rejected_pairs = set(), set()
for alt_bait, pep_grp in itt.groupby(sorted(alt_peptides), key=lambda x: x[:2]):
    pep_set = set([x[-1] for x in pep_grp])
    cnt_pep = len(pep_set)
    cnt_rejected = len(pep_set.intersection(nexprot_rejects))
    all_pairs.add(alt_bait)
    if cnt_rejected == cnt_pep:
        if 'UBBP4' in alt_bait[0]:
            print(pep_set.intersection(nexprot_rejects), pep_set)
        rejected_pairs.add(alt_bait)
print('all pairs:', len(all_pairs))
print('rejected:', len(rejected_pairs))

all pairs: 1148
rejected: 7


In [8]:
all_alt_psms = [x for x in all_alt_psms if x[0] not in nexprot_rejects]
pickle.dump(all_alt_psms, open('all_alt_psms.pkl', 'wb'))

In [9]:
reports_all_alts = {}
for bait, exps in bioplex_psms.items():
    for exp_id, psms in exps['exps'].items():
        for protgrp_psms in psms['protgrp_psms_valid_clean'].values():
            for prot_acc, gene in protgrp_psms['prot_accs_genes']:
                if is_alt(prot_acc):
                    couple = ('|'.join([gene, prot_acc]), bait)
                    if couple in rejected_pairs: continue
                    if prot_acc not in reports_all_alts:
                        reports_all_alts[prot_acc] = {'baits':set(), 'exp_ids':set(), 'psms':[], 'prot_group':[], 'gene':gene}
                    reports_all_alts[prot_acc]['baits'].add(bait)
                    reports_all_alts[prot_acc]['exp_ids'].add(exp_id)
                    reports_all_alts[prot_acc]['psms'].extend(protgrp_psms['psms'])
                    reports_all_alts[prot_acc]['prot_group'].extend(protgrp_psms['prot_accs_genes'])

pickle.dump(reports_all_alts, open('reports_all_alts.pkl', 'wb'))

In [10]:
reports_alts = {}
for bait, exps in bioplex_psms.items():
    for exp_id, psms in exps['exps'].items():
        for prot_acc, psms_dict in psms['psms'].items():
            if is_alt(prot_acc):
                if (prot_acc, bait) in rejected_pairs: continue
                gene, prot_acc = prot_acc.split('|')
                if prot_acc not in reports_alts:
                    reports_alts[prot_acc] = {'baits':set(), 'exp_ids':set(), 'psms':[], 'prot_group':[], 'gene':gene}
                reports_alts[prot_acc]['baits'].add(bait)
                reports_alts[prot_acc]['exp_ids'].add(exp_id)
                reports_alts[prot_acc]['psms'].extend(psms_dict['psms'])
pickle.dump(reports_alts, open('reports_alts.pkl', 'wb'))

In [11]:
for alt_acc, psm_dict in reports_alts.items():
    psm_dict['pep_bait_rep_cnts'] = get_pep_bait_rep_cnts(psm_dict['psms'])
    psm_dict['bait_rep_cnts']     = Counter([rep_bait_dict[exp] for exp in list(psm_dict['exp_ids'])])
    psm_dict['bait_peps']         = get_bait_peps(psm_dict['psms'])

In [12]:
alt_detected_pep_cnt = dict([(alt_acc, len(set(pep for pep, spec in d['psms']))) for alt_acc, d in reports_all_alts.items() if len(d['exp_ids']) >= len(d['baits'])*2])
pickle.dump(alt_detected_pep_cnt, open('bioplex_altprot_pep_cnt.pkl', 'wb'))

alt_detected_peps = [(alt_acc, set(pep for pep, spec in d['psms'])) for alt_acc, d in reports_all_alts.items() if len(d['exp_ids']) >= len(d['baits'])*2]

In [31]:
# PepQuery notebook ...

In [13]:
psms_by_prots = pickle.load(open('BioPlex_PepQ_psms_by_prots.pkl', 'rb'))

pepq_alts = []
validated_alt_baits = {}
pepq_validated_edges = set()
for alt_acc, psms in psms_by_prots.items():
    gene = prot_gene_dict[alt_acc]
    reps_per_bait = {}
    for bait, bait_reps in itt.groupby(sorted([(psm['bait'], psm['exp_name']) for psm in psms]), key=lambda x: x[0]):
        reps_per_bait[bait] = list(set([x[1] for x in bait_reps]))
    for bait in [k for k,v in reps_per_bait.items() if len(v)>=1]:
        if alt_acc in validated_alt_baits:
            validated_alt_baits[alt_acc][bait] = reps_per_bait[bait]
        else:
            validated_alt_baits[alt_acc] = {bait:reps_per_bait[bait],}
        pepq_alts.append(alt_acc)
        alt_gene_acc = '|'.join([alt_acc, gene])
        pepq_validated_edges.add(frozenset((bait, alt_gene_acc)))
pepq_alts = list(set(pepq_alts))
print('{} edges involving {} alts have been validated by PepQuery.'.format(len(pepq_validated_edges), len(pepq_alts)))

552 edges involving 280 alts have been validated by PepQuery.


In [14]:
psm_fields = ['exp_name', 'spectrum_title', 'peptide', 'modification', 'score', 'is_tryptic', 'include_anyway']
pepq_val_summary = {}
for alt_acc, psms in psms_by_prots.items():
    gene = prot_gene_dict[alt_acc]
    pepq_val_summary[alt_acc] = {}
    if alt_acc in validated_alt_baits: # alts will only be in here if they have been validated, otherwise no entry in that dict..
        baits = validated_alt_baits[alt_acc]
        for psm in psms:
            bait = psm['bait']
            if bait in baits:
                if bait not in pepq_val_summary[alt_acc]:
                    pepq_val_summary[alt_acc][bait] = []
                    
                spec_peps = set((x['spectrum_title'], x['peptide']) for x in pepq_val_summary[alt_acc][bait])
                if (psm['spectrum_title'], psm['peptide']) in spec_peps: continue
                val_psm = dict(zip(psm_fields, (psm[k] for k in psm_fields)))    
                ptms_scores = [(x['modification'], x['score']) for x in pepq_val_summary[alt_acc][bait] if x['spectrum_title']==psm['spectrum_title'] and psm['peptide']==x['peptide']]
                if ptms_scores:
                    ptm, score = sorted(ptms_scores, key=lambda x: -x[1])[0]
                    val_psm['modification'] = ptm
                    val_psm['score'] = score
                pepq_val_summary[alt_acc][bait].append(val_psm)

In [15]:
# wirte altprot identifications
detection_summary = []
with open('/home/xroucou_group/echange_de_fichiers/bioplex/bioplex_alt_prot_detection_summary.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['prey', 'gene', 'bait', 'experiment id', 'spectrum title', 'PepQ validated'])
    for alt_acc, d in reports_all_alts.items():
        if len(d['exp_ids']) >= len(d['baits'])*2:
            for pep_seq, spec in d['psms']:
                exp_id = spec.split('.')[0]
                bait = rep_bait_dict[exp_id]
                pepqval = False
                if alt_acc in pepq_val_summary and bait in pepq_val_summary[alt_acc]:
                    if any([(x['peptide']==pep_seq and x['exp_name']==exp_id) for x in pepq_val_summary[alt_acc][bait]]):
                        pepqval = True
                writer.writerow([alt_acc, d['gene'], bait, exp_id, spec, pepqval])
                detection_summary.append([alt_acc, d['gene'], bait, exp_id, spec, pepqval])
pickle.dump(detection_summary, open('detection_summary.pkl', 'wb'))

In [16]:
comppass_scores = {}
with open('BP2_OP16_features.csv', 'r') as f:
    reader = csv.reader(f)
    for n, row in enumerate(reader):
        if n==0:
            cols = row
            continue
        feat = dict(zip(cols, row))
        comppass_scores[(feat['Bait'], feat['Prey'].split('|')[0])] = {
            'Z':feat['Z'],
            'WD':feat['WD'],
            'entropy':feat['Entropy']
        }

In [17]:
predictions = pickle.load(open('HCIP_pred_BioPlex_OP16.pkl', 'rb'))

clf_scores = {(x[1], x[2].split('|')[0]):x[-1][1] for x in predictions}

In [18]:
pseudo_parent_dict = pickle.load(open('pseudo_parent_dict.pkl', 'rb'))

In [19]:
all_pepq_peps = []
ev2 = []
for alt_acc, psm_dict in list(reports_alts.items()):
    for bait in psm_dict['baits']:
        peps = psm_dict['bait_peps'][bait]
        coverage = get_coverage(alt_acc, peps)
        non_nested_peps = get_nonnested_peps(alt_acc, peps, margin=2)
        peps_2reps = [pep for pep in list(peps) if psm_dict['pep_bait_rep_cnts'][pep][bait]>0]
        pepq_peps = set()
        if alt_acc in pepq_val_summary and bait in pepq_val_summary[alt_acc]:
            pepq_peps = set(psm['peptide'] for psm in pepq_val_summary[alt_acc][bait] if psm['peptide'] not in exclude_peptides)
            all_pepq_peps.extend(list(pepq_peps))
            
        bait_prey = (bait, alt_acc)
        if bait_prey in comppass_scores:
            Z, WD, s = [comppass_scores[bait_prey][score] for score in ['Z', 'WD', 'entropy']]
        else:
            Z, WD, s = [0]*3
            
        if bait_prey in clf_scores:
            clf_score = clf_scores[bait_prey]
        else:
            clf_score = 'NA'
        prey_gene = psm_dict['gene']
        ev2.append(
            {
                'Bait':bait, 
                'Prey':alt_acc.replace('IP_2287811', 'IP_243680'), 
                'Prey Gene':prey_gene,
                'alt-ref same gene': 1 if prey_gene == bait else 0,
                'alt-ref pseudogene': 1 if prey_gene in pseudo_parent_dict and pseudo_parent_dict[prey_gene]['parent'] == bait else 0,
                'Count Replicates':psm_dict['bait_rep_cnts'][bait], 
                'Count Total Unique Peptides':len(non_nested_peps), 
                'Total Unique Peptides':'|'.join(list(non_nested_peps)),
                'Detected Coverage':coverage,
                'Peptides Set Meets HPP PE1':1 if pepset_is_HPP(alt_acc, non_nested_peps) else 0,
                'Count Unique Peptides (min. 2 rep.)':len(peps_2reps), 
                'Count Unique Peptides Validated PepQuery':len(pepq_peps),
                'PepQ Peptides':'|'.join(list(pepq_peps)),
                'Z score':Z,
                'WD score':WD,
                'entropy':s,
                'HCIP classifier score':clf_score
            }
        )

In [20]:
cols = (
    'Bait',
    'Prey',
    'Prey Gene',
    'alt-ref same gene',
    'alt-ref pseudogene',
    'Count Replicates',
    'Count Total Unique Peptides',
    'Total Unique Peptides',
    'Detected Coverage',
    'Peptides Set Meets HPP PE1',
    'Count Unique Peptides Validated PepQuery',
    'PepQ Peptides',
    'Z score',
    'WD score',
    'entropy',
    'HCIP classifier score'
)
with open('bioplex_tables/Table_EV2.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(cols)
    for row in ev2:
        writer.writerow([row[col] for col in cols])

In [21]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

wb = Workbook()
dest_filename = 'bioplex_tables/Table_EV2.xlsx'
ws1 = wb.active
ws1.title = "EV2"
ws1.append(cols)
for n, row in enumerate(ev2):
    row_idx = n+2
    ws1.append([row[col] for col in cols])
    
wb.save(filename = dest_filename)

In [22]:
# count alts that meet HPP PE1
len(set([x['Prey'] for x in ev2 if x['Peptides Set Meets HPP PE1']>0]))

68

In [21]:
check_pepq = True
cnt_upep_dict = {}
comppass = []
for bait in bioplex_psms:
    for exp_id in bioplex_psms[bait]['exps']:
        for prey in bioplex_psms[bait]['exps'][exp_id]['psms']:
            psm_ls = bioplex_psms[bait]['exps'][exp_id]['psms'][prey]['psms']
            if 'KRT' in prey: continue # filter out kreatins
            if is_alt(prey):
                if check_pepq:
                    gene, alt_acc = prey.split('|')
                    if alt_acc not in pepq_val_summary \
                    or bait not in pepq_val_summary[alt_acc] \
                    or all(not pep['include_anyway'] for pep in pepq_val_summary[alt_acc][bait]):
                        continue
                
            cnt_psms = len(bioplex_psms[bait]['exps'][exp_id]['psms'][prey]['psms'])
            if cnt_psms<1:continue
            cnt_upep_dict[(bait, prey, exp_id)] = bioplex_psms[bait]['exps'][exp_id]['psms'][prey]['n_upep']
            # Experiment.ID,Replicate,Experiment.Type,Bait,Prey,Spectral.Count
            comppass.append((bait_num_dict[bait], exp_id, 'True', bait, prey, cnt_psms))

In [22]:
len(set(x[4] for x in comppass if is_alt(x[4])))

263

In [24]:
len(set(x[4] for x in comppass if not is_alt(x[4])))

8972

In [16]:
# write full table for CompPASS
with open('BioPlex_psms_comppass.csv', 'w') as f:
    writer = csv.writer(f, )
    writer.writerow(('Experiment.ID', 'Replicate', 'Experiment.Type', 'Bait', 'Prey', 'Spectral.Count'))
    for putative_partners in comppass:
        writer.writerow(putative_partners)